# NBA Win Probability Model — Training Pipeline

This notebook contains the complete end-to-end pipeline used to build the NBA Win Probability Model, including data ingestion, data engineering, feature creation, model training, calibration, and evaluation. It produces the final neural-network model and calibration objects used in the real-time win probability generator.

---

## 📦 Overview

This notebook:

1. **Loads play-by-play and betting data**  
   - Aggregates multi-season PBP files  
   - Loads sportsbook moneylines  
   - Merges PBP events with pregame and in-game betting information  

2. **Cleans and standardizes raw data**  
   - Fixes incorrect or missing score fields  
   - Forward-fills score and possession state  
   - Removes malformed or outlier sportsbook lines  
   - Corrects time representations and handles overtime periods  

3. **Constructs game-state features**  
   - `seconds_remaining_game` (regulation + OT)  
   - `score_diff`, `score_home`, `score_away`  
   - Possession indicators  
   - Free throw remaining indicators  
   - Clutch markers  
   - Log and inverse time transforms (`t_log`, `t_inv`)  

4. **Integrates betting-derived priors**  
   - Converts moneylines to implied probabilities  
   - Averages across books  
   - Attaches pregame implied win probability to every state  
   - Cleans out extreme or erroneous sportsbook entries  

5. **Builds the final training dataset**  
   - One row per game state  
   - Feature columns defined consistently  
   - Label = final game outcome (home win = 1, away win = 0)  
   - Optional grouping to prevent leakage across games  

6. **Scales and splits the data**  
   - Normalizes continuous features  
   - Train/validation split using `GroupShuffleSplit`  
   - Ensures full games stay together in each partition  

7. **Defines and trains the neural network model**  
   - Custom `MLP` architecture in PyTorch  
   - Trains with Adam and BCE loss  
   - Uses validation metrics to tune training length  

8. **Calibrates the model**  
   - Wraps raw model outputs using Isotonic Regression  
   - Produces better-calibrated probabilities  
   - Saves auxiliary objects (`scaler`, `iso`, `feature_cols`)  

9. **Evaluates model quality**  
   - Brier score  
   - Log loss  
   - ROC AUC  
   - Probability histograms  
   - Reliability and calibration curves  

10. **Exports artifacts for inference notebook**  
    - `nn_winprob_model.pt` (trained PyTorch model)  
    - `nn_winprob_aux.pkl` (scaler, calibrator, feature list)  

---

## 📁 Output of This Notebook

After running the full pipeline, the following artifacts are produced:

- **`nn_winprob_model.pt`**  
  Trained PyTorch state dict for the MLP model.

- **`nn_winprob_aux.pkl`**  
  Contains:
  - `scaler` (StandardScaler fit on training data)  
  - `iso` (IsotonicRegression calibrator)  
  - `feature_cols` (ordered list of model input features)  

These files are consumed by the separate **single-game win probability generator** notebook.

---

## 🧠 Purpose of This Notebook

This file is the authoritative reference for how the win probability model is engineered and trained. It contains:

- All transformations  
- All model logic

In [ ]:
# === Data Handling ===
import pandas as pd
import numpy as np

# === Machine Learning / Modeling ===
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import brier_score_loss, log_loss, roc_auc_score

# === NBA API ===
from nba_api.stats.endpoints import PlayByPlayV3, boxscoresummaryv2

# === Utilities ===
import os
import sys
import time

## ⚠️ Data Download Notes

The first section of this notebook downloads all required play-by-play and game summary data from the NBA Stats API. This step **takes a very long time** and, due to API rate limiting, **cannot be completed in a single uninterrupted run**. After approximately 600 requests, the NBA API typically stops responding or begins returning errors.

To work around this, the download process is intentionally designed to:
- fetch data in **batches**,  
- **resume** from partially completed progress, and  
- **cache** previously downloaded files to avoid repeated API calls.

If you already have the cached data available, you can safely skip the download step and proceed directly to data engineering. Make sure to edit directories to match.

This file only downloads games that have betting data available. This data was collected from Kaggle. Both nba_betting_money_line.csv and nba_games_all.csv are needed for the project, but only the former is needed to download the games. 
 - https://www.kaggle.com/datasets/ehallmar/nba-historical-stats-and-betting-data/discussion


In [ ]:
# ---------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------

BASE_DIR = os.path.dirname(os.path.abspath("."))
ODDS_CSV_PATH = os.path.join(BASE_DIR, "csvs", "nba_betting_money_line.csv") # Directory to fetch betting odds CSV
PBP_OUT_DIR = os.path.join(BASE_DIR, "pbp_raw") # Directory to save raw PBP data
PROGRESS_START = 0  # For resuming partial runs
SLEEP_SECONDS = 0.25   # Gentle rate limit padding

# For testing: set to an int (e.g. 20). For full run: set to None.
DOWNLOAD_LIMIT = None


c:\Users\mvgai\SportsAnalytics\csvs\nba_betting_money_line.csv


In [ ]:
# ---------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------

def load_unique_game_ids(csv_path: str) -> list[str]:
    """Load and normalize unique game IDs from the odds CSV."""
    df = pd.read_csv(csv_path)

    if "game_id" in df.columns:
        col = "game_id"
    elif "gameId" in df.columns:
        col = "gameId"
    else:
        raise ValueError("Could not find game_id or gameId column in odds CSV.")

    ids = (
        df[col]
        .dropna()
        .astype(int)
        .astype(str)
        .map(lambda x: x.zfill(10))
        .unique()
    )
    id_list = list(ids)
    return id_list[PROGRESS_START:]


def ensure_output_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def pbp_output_path(game_id: str) -> str:
    # Save as CSV (no parquet engine needed)
    return os.path.join(PBP_OUT_DIR, f"{game_id}.csv")


def download_single_game(game_id: str) -> bool:
    """
    Download one game's PBP. Returns True on success, False on failure.
    """
    out_path = pbp_output_path(game_id)

    if os.path.exists(out_path):
        # already cached
        return True

    try:
        pbp = PlayByPlayV3(game_id=game_id)
        df = pbp.get_data_frames()[0]

        if df.empty:
            print(f"\n[WARN] Empty PBP for game {game_id}")
            return False

        # Write as CSV instead of parquet
        df.to_csv(out_path, index=False)
        return True

    except Exception as e:
        print(f"\n[ERROR] Failed for game {game_id}: {e}")
        return False

In [ ]:
# ---------------------------------------------------------------------
# Main - DO NOT RUN UNLESS READY TO DOWNLOAD DATA
# ---------------------------------------------------------------------

def main():
    start = time.time()
    print(f"Loading game IDs from {ODDS_CSV_PATH} ...")

    game_ids = load_unique_game_ids(ODDS_CSV_PATH)
    total = len(game_ids)
    print(f"Found {total} games.")

    ensure_output_dir(PBP_OUT_DIR)

    num_ok = 0
    num_fail = 0

    for idx, gid in enumerate(game_ids, start=1):
        if DOWNLOAD_LIMIT is not None and idx > DOWNLOAD_LIMIT:
            break

        # Single-line progress update
        progress = f"[{idx}/{total}] Downloading PBP for {gid} ..."
        sys.stdout.write("\r" + progress)
        sys.stdout.flush()

        success = download_single_game(gid)
        if success:
            num_ok += 1
        else:
            num_fail += 1

        time.sleep(SLEEP_SECONDS)

    # Final newline to clear the progress line
    print()
    end = time.time()
    elapsed = end - start
    print(f"Completed in {elapsed:.2f} seconds.")
    print("\n[SUMMARY]")
    print(f"  Total games considered:   {min(total, DOWNLOAD_LIMIT or total)}")
    print(f"  Successfully downloaded:  {num_ok}")
    print(f"  Failed / errors:          {num_fail}")
    print(f"  Output directory:         {PBP_OUT_DIR}")

## 🏷️ Betting Odds Standardization

The sportsbook odds dataset is not consistent in how it labels the **home** and **away** teams across different entries. Some rows list the home team in the first column, others label teams in reverse order, and a few books use their own internal formatting.

This section cleans and standardizes the odds data by:
- correctly identifying which team is the **home** team for each game,  
- aligning all rows to a consistent `(home_team, away_team)` format,  
- ensuring moneylines always correspond to the correct team, and  
- producing unified `home_win_implied_prob` and `away_win_implied_prob` fields.

After this step, all betting information is aligned with the PBP data and ready for feature engineering.

In [ ]:
betting_data = pd.read_csv("csvs/nba_betting_money_line.csv")
games_all = pd.read_csv("csvs/nba_games_all.csv")

In [ ]:
# Step 1: Filter rows where 'matchup' contains an '@' sign using regex
filtered_data = games_all[games_all['matchup'].str.contains(r'@', regex=True, na=False)]

In [ ]:
# Adjust home and away teams in betting data if needed

home_away_dict = games_all.set_index('game_id')[['team_id', 'is_home']].apply(tuple, axis=1).to_dict()

# Step 3: Define a function to adjust team_id and a_team_id based on 'is_home'
def adjust_team_and_prices(row):
    # Get the game_id for the current row
    game_id = row['game_id']
    team_id = row['team_id']
    # Get the is_home value from the dictionary
    is_home = home_away_dict.get(game_id)[1]
    d_team_id = home_away_dict.get(game_id)[0]
    
    if team_id == d_team_id and is_home == 'f':
        print("swapping for game_id:", game_id)
        row['team_id'], row['a_team_id'] = row['a_team_id'], row['team_id']
        row['price1'], row['price2'] = row['price2'], row['price1']
           
    if team_id != d_team_id and is_home == 't':
        print("swapping for game_id:", game_id)
        row['team_id'], row['a_team_id'] = row['a_team_id'], row['team_id']
        row['price1'], row['price2'] = row['price2'], row['price1']
    
    # Return the adjusted row
    return row

# Step 4: Apply the adjustment function to each row in the betting data
betting_data_adjusted = betting_data.apply(adjust_team_and_prices, axis=1)

In [ ]:
# Adjust if needed
ADJUSTED_ODDS_CSV_PATH = os.path.join(BASE_DIR, "csvs", "adjusted_betting_data.csv")
betting_data_adjusted.to_csv(ADJUSTED_ODDS_CSV_PATH, index=False)

## Data Cleaning and Game State Creation

### 🧹 Data Cleaning

This section prepares the raw play-by-play and odds data so it can be reliably used for feature engineering and model training. The NBA Stats API often returns incomplete or irregular fields (such as missing scores, empty strings, inconsistent possession markers, or duplicated events), and the betting dataset includes occasional malformed or extreme values.

Key cleaning steps include:
- fixing missing or blank score fields using forward-fill logic,  
- correcting inconsistent possession or event markers,  
- reconciling regulation and overtime time formats,  
- removing or capping extreme sportsbook outliers, and  
- ensuring all numeric fields are properly typed and aligned.

After this cleaning stage, the dataset has consistent structure and values, making downstream feature construction and modeling stable and reproducible.

### 🔄 Parsing Play-by-Play into Game States

Raw play-by-play data from the NBA Stats API arrives as a long list of individual events—shots, turnovers, rebounds, fouls, substitutions, etc. While this format is useful for descriptive analysis, it is **not directly usable** for win probability modeling. The model requires a unified representation of the game at each moment in time.

In this section, the event-level PBP data is transformed into a sequence of **game states**, where each state represents a snapshot of the game after every meaningful update (score change, possession change, or time change). This process involves:

- extracting scores, possession, period, and clock at each event,  
- computing `seconds_remaining_game` across regulation and overtime,  
- carrying forward score and possession information between events,  
- identifying meaningful transitions (e.g., new possession, score update), and  
- constructing one clean row per game state with synchronized fields.

The result is a structured, chronological timeline of the game that the model can interpret—each row holding all features needed to estimate the probability of the home team winning at that specific moment.


In [ ]:
def fix_scores_carry_forward(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each game, carry forward the last non-zero scoreHome/scoreAway.
    Assumes df is already sorted in chronological PBP order within each game.
    Recomputes score_diff at the end.
    """
    df = df.copy()
    score_cols = ["scoreHome", "scoreAway"]  # or whatever your score columns are

    for col in score_cols:
        # Convert to numeric, turn '' and other junk into NaN
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
        # Forward fill within the game, then fill any leading NaNs with 0
        df[col] = df[col].ffill().fillna(0).astype(int)
    # Decide how to group (per game or whole df)
    if "gameId" in df.columns:
        group_cols = ["gameId"]
    elif "game_id" in df.columns:
        group_cols = ["game_id"]
    else:
        group_cols = None  # assume df is already a single game

    def _fix_one_game(g: pd.DataFrame) -> pd.DataFrame:
        # If you want to be extra safe, you can sort inside each game
        # by period/time/event_num, e.g.:
        # sort_cols = [c for c in ["period", "event_num"] if c in g.columns]
        # if sort_cols:
        #     g = g.sort_values(sort_cols)

        for col in ["scoreHome", "scoreAway"]:
            if col in g.columns:
                g[col] = (
                    g[col]
                    .replace(0, np.nan)  # treat 0 as "missing" between scores
                    .ffill()             # carry forward last known score
                    .fillna(0)           # before first score, keep 0-0
                    .astype(int)
                )

        # Recompute score_diff if present / desired
        if "scoreHome" in g.columns and "scoreAway" in g.columns:
            g["score_diff"] = g["scoreHome"] - g["scoreAway"]

        return g

    if group_cols is None:
        df = _fix_one_game(df)
    else:
        df = df.groupby(group_cols, group_keys=False).apply(_fix_one_game)

    return df

In [ ]:
def american_to_implied_prob(odds):
    """
    Convert American odds to implied probability (no vig removal).
    """
    if pd.isna(odds):
        return np.nan

    odds = float(odds)

    if odds < 0:
        return (-odds) / ((-odds) + 100)
    else:
        return 100 / (odds + 100)

In [ ]:
def parse_iso_clock_to_seconds(clock_str) -> int:
    """
    Parse PlayByPlayV3 'clock' strings like 'PT11M54.00S' into seconds.
    """
    if clock_str is None or pd.isna(clock_str):
        return 0

    s = str(clock_str).strip()
    if not s.startswith("PT"):
        return 0

    # Strip 'PT' and trailing 'S'
    s = s[2:]
    if s.endswith("S"):
        s = s[:-1]

    # Split on 'M'
    if "M" not in s:
        return 0

    minutes_str, seconds_str = s.split("M", 1)

    try:
        minutes = int(float(minutes_str))
    except ValueError:
        minutes = 0

    try:
        seconds = int(float(seconds_str))
    except ValueError:
        seconds = 0

    return minutes * 60 + seconds


def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add:
      - seconds_remaining_in_period: time left in the current period
      - seconds_remaining_game: time left until this period ends, assuming
        the game ends at 0:00 of this period (i.e., end of Q4 or an OT)

    For periods 1–4, seconds_remaining_game is "time left in regulation".
    For periods > 4 (OTs), seconds_remaining_game is just the clock time left
    in that OT period.
    """
    df = df.copy()

    # 1) Seconds remaining in current period from the ISO clock
    df["seconds_remaining_in_period"] = df["clock"].apply(parse_iso_clock_to_seconds)

    period_length = 12 * 60        # 12-minute quarters
    total_reg = 4 * period_length  # 48 minutes

    def compute_seconds_remaining_game(row):
        period = row["period"]
        sec_left = row["seconds_remaining_in_period"]

        if period <= 4:
            # Time elapsed before this period
            elapsed_before = (period - 1) * period_length
            # Time elapsed within this period
            elapsed_in_period = period_length - sec_left
            elapsed_total = elapsed_before + elapsed_in_period
            # Time left in regulation
            return max(0, total_reg - elapsed_total)
        else:
            # In any OT, treat "time remaining until game ends" as just
            # the time left in this OT period.
            return sec_left

    df["seconds_remaining_game"] = df.apply(compute_seconds_remaining_game, axis=1)

    return df

In [ ]:
def add_score_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure scores are numeric, forward-filled, and add score_diff (home - away).
    """
    df = df.copy()

    # Some rows may have None / NaN scores; convert and forward fill.
    df["scoreHome"] = pd.to_numeric(df["scoreHome"], errors="coerce")
    df["scoreAway"] = pd.to_numeric(df["scoreAway"], errors="coerce")

    df["scoreHome"] = df["scoreHome"].ffill().fillna(0).astype(int)
    df["scoreAway"] = df["scoreAway"].ffill().fillna(0).astype(int)

    df["score_diff"] = df["scoreHome"] - df["scoreAway"]

    return df

In [ ]:
def add_outcome_label(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add a binary label 'home_win' for each row based on final score.
    """
    df = df.copy()

    # Look at the last non-null score row to determine result
    final_home = df["scoreHome"].iloc[-1]
    final_away = df["scoreAway"].iloc[-1]

    home_win = int(final_home > final_away)
    df["home_win"] = home_win

    return df

In [ ]:
def get_home_away_team_ids(game_id: str):
    summary = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id)
    game_summary = summary.game_summary.get_data_frame().iloc[0]

    home_team_id = int(game_summary["HOME_TEAM_ID"])
    away_team_id = int(game_summary["VISITOR_TEAM_ID"])
    return home_team_id, away_team_id

In [ ]:
def parse_free_throw_subtype(sub_type: str):
    """
    Parse subType strings like:
      'Free Throw 1 of 2'
      'Free Throw 2 of 2'
      'Free Throw Flagrant 1 of 2'
      'Free Throw Flagrant 2 of 2'

    Returns:
      (idx, total) as ints, or (None, None) if no match.

    If we later encounter patterns like 'Free Throw Technical' with no
    'X of Y', we can treat them as (1,1).
    """
    if not isinstance(sub_type, str):
        return None, None

    s = sub_type.strip()
    s = re.sub(r"\s+", " ", s)  # normalize whitespace

    # General pattern: "Free Throw [optional word(s)] X of Y"
    # e.g. "Free Throw 1 of 2", "Free Throw Flagrant 1 of 2"
    m = re.search(r"Free Throw.*?(\d+)\s+of\s+(\d+)", s, flags=re.IGNORECASE)
    if m:
        idx = int(m.group(1))
        total = int(m.group(2))
        return idx, total

    # Fallback: something like "Free Throw Technical" → treat as 1 of 1
    if "free throw" in s.lower():
        return 1, 1

    return None, None

In [ ]:
def _loc_to_sign(loc_value: str) -> int:
    """
    Map PlayByPlayV3 'location' field to +1 (home), -1 (away), or 0 (neutral).

    Adjust this mapping if your actual 'location' values differ.
    """
    if loc_value is None or pd.isna(loc_value):
        return 0
    s = str(loc_value).strip().lower()

    # Common patterns: 'home', 'away', but tweak if you see others
    if s in ("home", "hom", "h"):
        return 1
    if s in ("away", "awy", "visitor", "vis", "v"):
        return -1
    return 0

def add_free_throw_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add:
      - home_ft_remaining
      - away_ft_remaining

    Semantics: at each row, these give the number of free throws still to be
    taken by each team *after* this event.

    Logic:
      - Identify free throw rows by actionType containing 'Free Throw'.
      - Parse subType ('Free Throw 1 of 2', 'Free Throw Flagrant 1 of 2', etc.)
      - remaining = max(0, total - idx)
      - If the shooter is home:  home_ft_remaining = remaining
      - If the shooter is away: away_ft_remaining = remaining
      - Non-FT rows keep the previous remaining values.
    """
    df = df.copy()

    # Ensure we have location_sign
    if "location_sign" not in df.columns:
        df["location_sign"] = df["location"].apply(_loc_to_sign)

    action_types = df["actionType"].fillna("").astype(str)

    home_ft_rem = 0
    away_ft_rem = 0

    home_ft_remaining = []
    away_ft_remaining = []

    for i in range(len(df)):
        atype = action_types.iat[i].lower()
        loc_sign = df["location_sign"].iat[i]

        # If this row is a free throw, update the counters based on subType
        if "free throw" in atype:
            sub_type = df["subType"].iat[i]
            idx, total = parse_free_throw_subtype(sub_type)

            if idx is not None and total is not None:
                remaining = max(0, total - idx)
                if loc_sign == 1:
                    # Home shooting FTs
                    home_ft_rem = remaining
                elif loc_sign == -1:
                    # Away shooting FTs
                    away_ft_rem = remaining
                # If loc_sign == 0, we ignore (weird neutral FT case)
        # For non-FT rows, we just carry over current home_ft_rem / away_ft_rem

        # Record the state *after* this event
        home_ft_remaining.append(home_ft_rem)
        away_ft_remaining.append(away_ft_rem)

    df["home_ft_remaining"] = home_ft_remaining
    df["away_ft_remaining"] = away_ft_remaining

    # You can keep or drop location_sign depending on whether you use it elsewhere
    # df.drop(columns=["location_sign"], inplace=True)

    return df

def add_possession_after(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add a 'possession' column representing who has the ball AFTER each event,
    from the home team's perspective:

        possession ∈ {+1, 0, -1}
          +1 = home has ball
          -1 = away has ball
           0 = no clear possession (end of period, jump ball, game over, etc.)

    Logic:
      - Look ahead from each row to the next relevant action:
          * Skip Substitution / Timeout rows
          * If next is Foul: invert the location team
          * If next is period / Jump Ball: set possession = 0
          * Else: possession = location of that next row (home/away)
      - For rows that are themselves period / Jump Ball: possession = 0
    """
    df = df.copy()

    # Precompute a simple sign for each row's location
    df["location_sign"] = df["location"].apply(_loc_to_sign)

    n = len(df)
    possession_after = [0] * n

    action_types = df["actionType"].fillna("").astype(str)

    for i in range(n):
        atype_i = action_types.iat[i].lower()

        # If this row itself is end-of-period or a jump ball, possession is 0
        if "period" in atype_i or "jump ball" in atype_i:
            possession_after[i] = 0
            continue
        if "rebound" in atype_i:
            possession_after[i] = df["location_sign"].iat[i]
            continue
        # Look ahead to find the next relevant offensive action
        poss = 0
        j = i + 1
        while j < n:
            atype_j = action_types.iat[j].lower()
            loc_sign_j = df["location_sign"].iat[j]

            # Skip subs/timeouts when searching
            if "substitution" in atype_j or "timeout" in atype_j:
                j += 1
                continue

            # If the next significant event is end-of-period or jump ball → no possession
            if "period" in atype_j or "jump ball" in atype_j:
                poss = 0
                break

            # If the next event is a foul, invert the fouling team to get offense
            if "foul" in atype_j:
                if loc_sign_j == 1:
                    poss = -1
                elif loc_sign_j == -1:
                    poss = 1
                else:
                    poss = 0
                break

            # Otherwise, treat the next event's team as having the ball
            if loc_sign_j != 0:
                poss = loc_sign_j
                break

            # If location is neutral/unknown, keep scanning
            j += 1

        possession_after[i] = poss

    df["possession"] = possession_after
    # Optionally drop helper
    df.drop(columns=["location_sign"], inplace=True)

    return df

In [ ]:
def filter_to_state_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep only rows where we want to define a model state.

    States are:
      - After a MADE field goal
      - After a rebound (except 'Normal Rebound' pseudo-events)
      - After a turnover
      - After dead-ball / possession-setting events (fouls, timeouts, violations, OOB, etc.)
      - During free throws that are NON-REBOUNDABLE on a miss
        (all pre-final FTs in a trip, plus flagrant/technical FTs)
      - During reboundable FTs *only if made* (missed ones defer to the rebound row)

    We DROP:
      - Missed normal field goals (state comes at rebound/dead-ball)
      - Missed reboundable FTs (state comes at rebound)
      - Duplicate offensive-foul rows
      - Fake 'Normal Rebound' rows after non-reboundable FTs
    """
    df = df.copy()

    def _is_state_row(row) -> bool:
        atype = str(row["actionType"]).lower()
        sub_raw = row.get("subType", "")
        sub = str(sub_raw).strip()
        poss  = row["possession"]
        sub_type = str(sub_raw)
        sub_lower = sub_type.lower()
        if atype == "made shot":
            return True
        # 1) Remove offensive-charge duplicate foul rows
        if sub == "Offensive Charge":
            return False

        # 2) Remove fake team rebounds after non-reboundable missed FTs
        if sub == "Normal Rebound":
            return False

        # Only keep rows where we actually know who has the ball afterward
        if poss == 0:
            return False

        desc = str(row.get("description", "") or "")
        desc_upper = desc.upper()

        # --- FREE THROWS ---
        if "free throw" in atype:

            # Parse X of Y if present (normal FT trips)
            idx, total = parse_free_throw_subtype(sub_type)

            # Flagrant / technical FTs: treat as NON-REBOUNDABLE → always keep
            if "flagrant" in sub_lower or "technical" in sub_lower:
                return True

            # Normal FTs with 'X of Y'
            if idx is not None and total is not None:
                if idx < total:
                    # Not the last FT in the trip → NON-REBOUNDABLE (dead ball regardless)
                    return True
                else:
                    # Last FT in a normal trip → REBOUNDABLE
                    # Use description: if it starts with 'MISS', it's a miss.
                    is_miss = desc_upper.startswith("MISS")
                    if not is_miss:
                        # Made last FT → define a state here
                        return True
                    else:
                        # Missed last FT → let the rebound/dead-ball event define the state
                        return False

            # If we can't parse, safest assumption is: non-reboundable FT → keep
            return True

        # --- FIELD GOALS (NON-FT) ---
        is_fg_row = bool(row.get("isFieldGoal", 0))
        sr = str(row.get("shotResult", "")).lower()

        if is_fg_row:
            # For non-FT FGs we can still use shotResult; it's usually reliable.
            if "made" in sr:
                return True          # made FG → state here
            if "miss" in sr:
                return False         # missed FG → state at rebound
            return False             # unknown, drop

        # --- REBOUNDS: possession-resolving event ---
        if "rebound" in atype:
            return True

        # --- TURNOVERS: possession-resolving event ---
        if "turnover" in atype:
            return True

        # --- DEAD-BALL / POSSESSION-SETTING EVENTS ---
        if "timeout" in atype:
            return True

        # Will be cross-listed as turnover if relevant
        if "violation" in atype:
            return False

        if "out of bounds" in atype or "out-of-bounds" in atype:
            return True

        # Fouls often lead to FTs or side-out; possession is effectively known
        if "foul" in atype:
            if "offensive" in sub_lower:
                return False
            return True

        # Period/jump ball: ambiguous/neutral → skip as states
        if "period" in atype or "jump ball" in atype:
            return False
        
        # Everything else: non-state by default
        return False

    mask = df.apply(_is_state_row, axis=1)
    return df[mask].reset_index(drop=True)

In [ ]:
def add_clutch_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add a boolean 'is_clutch' column.

    Definition:
      - True for all OT periods (period > 4).
      - In regulation (period <= 4), true if:
          * period == 4
          * seconds_remaining_in_period <= 300 (last 5 minutes of 4Q)
          * abs(score_diff) <= 5
    """
    df = df.copy()

    # Make sure needed columns exist
    if "seconds_remaining_game" not in df.columns:
        raise ValueError("seconds_remaining_game column is required")
    if "score_diff" not in df.columns:
        raise ValueError("score_diff column is required")

    period = df["period"]
    sec_left = df["seconds_remaining_game"]
    score_diff = df["score_diff"].astype(float)

    # OT: always clutch
    in_ot = period > 4

    # Regulation clutch: last 5 min of 4Q and close score
    in_reg_clutch = (
        (period == 4)
        & (sec_left <= 300.0)
        & (np.abs(score_diff) <= 5.0)
    )

    df["is_clutch"] = (in_ot | in_reg_clutch).astype(int)  # or keep as bool

    return df

In [ ]:
def build_game_state_dataset_for_game(df: pd.DataFrame) -> pd.DataFrame:
    df = fix_scores_carry_forward(df)
    df = add_time_features(df)
    df = add_score_features(df)
    df = add_possession_after(df)      # lookahead-based, training-time only
    df = add_free_throw_features(df)
    df = add_outcome_label(df)
    df = add_clutch_feature(df)
    

    df = filter_to_state_rows(df)      # <-- drop missed-shot rows, etc.
    cols = [
        "gameId",
        "period",
        "clock",
        "seconds_remaining_game",
        "is_clutch",
        "scoreHome",
        "scoreAway",
        "score_diff",
        "possession",
        "home_ft_remaining",
        "away_ft_remaining",
        "home_win",
        "actionType",
        "subType",
        "location",
        "description",
        "shotResult",
        "isFieldGoal",
    ]
    cols = [c for c in cols if c in df.columns]
    return df[cols].copy()

In [ ]:
def load_pbp_v3(game_id: str) -> pd.DataFrame:
    path = os.path.join(PBP_OUT_DIR, f"{game_id}.csv") # Same as directory to save raw PBP data
    print("Loading:", path)
    return pd.read_csv(path)

In [ ]:
def build_processed_odds(odds_csv_path: str) -> pd.DataFrame:
    raw = pd.read_csv(odds_csv_path)
    raw["z1"] = raw.groupby("game_id")["price1"].transform(
        lambda x: (x - x.mean()) / x.std()
    )
    raw["z2"] = raw.groupby("game_id")["price2"].transform(
        lambda x: (x - x.mean()) / x.std()
    )
    raw = raw[raw["z1"].abs() < 2]
    raw = raw[raw["z2"].abs() < 2]
    raw.drop(['z1','z2'], axis=1, inplace=True)
    
    if "game_id" in raw.columns:
        game_col = "game_id"
    elif "gameId" in raw.columns:
        game_col = "gameId"
    else:
        raise ValueError("Could not find game_id or gameId column in odds CSV.")

    df = raw.rename(columns={
        game_col: "gameId",
        "price1": "home_ml",
        "price2": "away_ml",
    })

    grouped = df.groupby("gameId").agg({
        "home_ml": "mean",
        "away_ml": "mean",
    }).reset_index()

    grouped["implied_prob_home"] = grouped["home_ml"].apply(american_to_implied_prob)
    grouped["implied_prob_away"] = grouped["away_ml"].apply(american_to_implied_prob)
    grouped["gameId"] = (
        grouped["gameId"]
        .astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(10)
    )

    return grouped

In [ ]:
def list_cached_game_ids(pbp_dir: str) -> list[str]:
    game_ids = []
    for fname in os.listdir(pbp_dir):
        if not fname.lower().endswith(".csv"):
            continue
        game_id = os.path.splitext(fname)[0]
        game_ids.append(game_id)
    return sorted(game_ids)

def build_full_training_dataset_from_cache_serial(
    pbp_dir: str,
    odds_csv_path: str,
) -> pd.DataFrame:
    
    print("Using pbp_dir:", pbp_dir)
    print("Files in pbp_dir:", len(os.listdir(pbp_dir)))
    print(os.listdir(pbp_dir)[:10])  # first 10 entries
    
    processed_odds = build_processed_odds(odds_csv_path)
    
    
    cached_games = set(list_cached_game_ids(pbp_dir))
    games_with_odds = set(processed_odds["gameId"].astype(str))
    usable_games = sorted(cached_games & games_with_odds)

    print(f"[INFO] Cached PBP games:      {len(cached_games)}")
    print(f"[INFO] Games with odds:       {len(games_with_odds)}")
    print(f"[INFO] Usable intersection:   {len(usable_games)}")

    odds_subset = processed_odds[processed_odds["gameId"].isin(usable_games)]

    dfs = []
    for row in odds_subset.itertuples(index=False):
        game_id = row.gameId
        home_imp = row.implied_prob_home
        away_imp = row.implied_prob_away

        print(f"[GAME] Processing {game_id}...")

        try:
            states = build_game_state_dataset_for_game(game_id)
        except Exception as e:
            print(f"[WARN] Skipping game {game_id} due to error: {e}")
            continue

        states["home_win_implied_prob"] = home_imp
        states["away_win_implied_prob"] = away_imp

        dfs.append(states)

    if not dfs:
        raise RuntimeError("No games were successfully processed from cache (serial).")

    training_df = pd.concat(dfs, ignore_index=True)
    return training_df

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="DataFrameGroupBy.apply",
)

## 🧠 Model Creation and Training

This section builds and trains the neural-network win probability model using the engineered game-state dataset. The model is designed to estimate the probability that the home team wins the game given the score, time remaining, possession, free throws, betting priors, and other contextual features.

Key steps include:

- **Defining the model architecture:**  
  A lightweight multilayer perceptron (MLP) implemented in PyTorch, with nonlinear activations and dropout for regularization.

- **Preparing the training data:**  
  Scaling continuous variables, selecting the final feature set, and grouping rows by game to avoid leakage in cross-validation.

- **Training the model:**  
  Using the Adam optimizer with binary cross-entropy loss, monitoring validation performance, and tuning training duration to prevent overfitting.

- **Calibrating probabilities:**  
  Raw neural network outputs are passed through an Isotonic Regression calibration layer to produce accurate and well-behaved win probabilities.

- **Evaluating performance:**  
  Computing Brier score, log loss, and ROC AUC on validation data to assess both accuracy and calibration quality.

This stage produces the final trained model (`nn_winprob_model.pt`) and the auxiliary objects (`nn_winprob_aux.pkl`) that power the real-time win probability generator.

In [ ]:
# 1. Load training data
df = pd.read_csv("full_training_dataset.csv")

# ---- Adjust this if your column is named differently ----
TARGET_COL = "home_win"          # 1 if home wins, 0 if away wins
GAME_ID_COL = "gameId"           # or "game_id", etc.
# ---------------------------------------------------------

# Time transforms for better behavior
if "seconds_remaining_game" not in df.columns:
    raise ValueError("Expected 'seconds_remaining_game' in df for time-based features.")

df["t_log"] = np.log1p(df["seconds_remaining_game"])
df["t_inv"] = 1.0 / (1.0 + df["seconds_remaining_game"])

if GAME_ID_COL not in df.columns:
    raise ValueError(f"Expected a game id column '{GAME_ID_COL}' in df.columns")

if TARGET_COL not in df.columns:
    raise ValueError(f"Expected target column '{TARGET_COL}' in df.columns")

In [ ]:
# Time transforms for better behavior
if "seconds_remaining_game" not in df.columns:
    raise ValueError("Expected 'seconds_remaining' in df for time-based features.")

df["t_log"] = np.log1p(df["seconds_remaining_game"])          # smooth log time
df["t_inv"] = 1.0 / (1.0 + df["seconds_remaining_game"])      # high sensitivity for late game

In [ ]:
def compute_time_decay_weights(df: pd.DataFrame) -> pd.Series:
    """
    Higher weight for later-game states.
    Assumes df has a 'seconds_remaining' column (bigger = earlier in game).
    Optionally boosts 4th quarter if 'period' exists.
    """
    if "seconds_remaining_game" not in df.columns:
        raise ValueError("Need a 'seconds_remaining_game' column to compute time-decay weights.")

    t = df["seconds_remaining_game"].astype(float)

    # Scale so earliest times get weight ~1, latest get ~2
    t_max = t.max()
    w = 1.0 + (t_max - t) / t_max   # ranges roughly from 1.0 to 2.0

    # Optional: extra emphasis on 4th quarter and OT
    if "period" in df.columns:
        w = w * (1.0 + 0.5 * (df["period"] >= 4))

    return w

weights = compute_time_decay_weights(df)

In [ ]:
# 2. Define X, y
y = df[TARGET_COL].astype(int)
groups = df[GAME_ID_COL].astype(str)

# Drop non-feature columns (ids, text, etc.)
drop_cols = {TARGET_COL, GAME_ID_COL}

# Add anything else you *know* should not be a feature:
drop_cols |= {
    "event_id", "eventNum", "description", "actionType", "subType", "clock", "period", 'location', 'shotResult', 'isFieldGoal'
} & set(df.columns)

feature_cols = [c for c in df.columns if c not in drop_cols]

X = df[feature_cols]
print("Using", len(feature_cols), "features:")
print(feature_cols)

In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_idx, val_idx = next(gss.split(X, y, groups=groups))
w_train = weights.iloc[train_idx].to_numpy()
w_val   = weights.iloc[val_idx].to_numpy()  # usually not used, but available if needed

X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

print("Train rows:", len(X_train), "Val rows:", len(X_val))
print("Train games:", len(np.unique(groups.iloc[train_idx])),
      "Val games:", len(np.unique(groups.iloc[val_idx])))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 1) Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

# 2) PyTorch Dataset
class NNDataset(Dataset):
    def __init__(self, X, y, w=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        if hasattr(y, "values"):
            y = y.values
        self.y = torch.tensor(y, dtype=torch.float32)
        if w is None:
            self.w = torch.ones_like(self.y)
        else:
            if hasattr(w, "values"):
                w = w.values
            self.w = torch.tensor(w, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.w[idx]

train_ds = NNDataset(X_train_scaled, y_train, w_train)
val_ds   = NNDataset(X_val_scaled, y_val, np.ones_like(y_val))

train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=1024, shuffle=False)

# 3) Define the MLP
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1),  # logits
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)  # shape (batch,)

model = MLP(input_dim=X_train_scaled.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss(reduction="none")  # we'll apply weights manually

# 4) Training loop with sample weights + simple early stopping
best_val_loss = float("inf")
patience = 5
epochs_no_improve = 0
num_epochs = 40

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    n_train = 0

    for xb, yb, wb in train_loader:
        xb, yb, wb = xb.to(device), yb.to(device), wb.to(device)

        logits = model(xb)
        loss_raw = criterion(logits, yb)      # (batch,)
        loss = (loss_raw * wb).mean()        # apply sample weights

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * xb.size(0)
        n_train += xb.size(0)

    train_loss /= n_train

    # validation loss (unweighted, just for early stopping)
    model.eval()
    val_loss = 0.0
    n_val = 0
    with torch.no_grad():
        for xb, yb, wb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss_raw = criterion(logits, yb)
            loss = loss_raw.mean()
            val_loss += loss.item() * xb.size(0)
            n_val += xb.size(0)
    val_loss /= n_val

    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        epochs_no_improve = 0
        best_state = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping.")
            break

# Load best weights
model.load_state_dict(best_state)

# 5) Uncalibrated validation metrics
model.eval()
val_logits_list = []
with torch.no_grad():
    for xb, yb, wb in val_loader:
        xb = xb.to(device)
        logits = model(xb)
        val_logits_list.append(logits.cpu())

In [ ]:
val_logits = torch.cat(val_logits_list)
val_proba_raw = torch.sigmoid(val_logits).numpy().flatten()

print("Uncalibrated PyTorch NN metrics:")
print("  Brier:", brier_score_loss(y_val, val_proba_raw))
print("  Log loss:", log_loss(y_val, val_proba_raw))
print("  ROC AUC:", roc_auc_score(y_val, val_proba_raw))

In [ ]:
iso = IsotonicRegression(out_of_bounds="clip")

# Fit isotonic reg on val-set raw probs vs true labels
iso.fit(val_proba_raw, y_val.to_numpy())

val_proba_cal = iso.transform(val_proba_raw)

print("Calibrated PyTorch NN metrics:")
print("  Brier:", brier_score_loss(y_val, val_proba_cal))
print("  Log loss:", log_loss(y_val, val_proba_cal))
print("  ROC AUC:", roc_auc_score(y_val, val_proba_cal))

In [ ]:
# Edit file paths as needed

model_bundle = {
    "feature_cols": feature_cols,
    "game_id_col": GAME_ID_COL,
    "target_col": TARGET_COL,
    "model": val_proba_cal,
}
torch.save(model.state_dict(), "nn_winprob_model.pt")
aux_bundle = {
    "scaler": scaler,
    "iso": iso,
    "feature_cols": feature_cols,
}
joblib.dump(aux_bundle, "nn_winprob_aux.pkl")
joblib.dump(model_bundle, "nba_winprob_nn_calibrated.pkl")
print("Saved model to nba_winprob_nn_calibrated.pkl")